In [1]:
from ARIMA_FUNCTIONS_REAL import *

# S&P 500

In [2]:
# download data
df_SnP_ARIMA = get_data_ARIMA(index_symbol='^GSPC', start='2000-01-01', end = "2023-09-5")

[*********************100%***********************]  1 of 1 completed


### Hyperparameter Tunning SET-UP

In [3]:
lookback_bars = 1000
validation_bars = 250
testing_bars = 250
lag = 21

p_max = 6
q_max = 6

SENS = 'RANDOM_SEARCH_1'
PATH = F'S&P500/Sensitivity Analysis/{SENS}'

TRANSACTION_COST = (0.1/100)

# ARIMA

In [4]:
model_predictions_SnP, train_data_SnP, test_validation_data_SnP = main_arima(p_max, q_max, df_SnP_ARIMA, lookback_bars, validation_bars, testing_bars)

1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 250
1000 205


In [5]:
df_test_SnP_ARIMA = pd.concat(test_validation_data_SnP, axis=0)

In [6]:
num = lookback_bars+validation_bars+lag

inv_y_test_LO = df_test_SnP_ARIMA['Close'].values[validation_bars+lag:]
inv_y_pred_test_LO = model_predictions_SnP[validation_bars+lag:]

In [7]:
df_inv_y_test_LO = pd.DataFrame(data={'Date': df_SnP_ARIMA.index[num:(len(inv_y_test_LO)+num)], 'inv_y__test': inv_y_test_LO})
df_inv_y_pred_test_LO = pd.DataFrame(data={'Date': df_SnP_ARIMA.index[num:(len(inv_y_test_LO)+num)], 'inv_y_pred_test': inv_y_pred_test_LO})

In [8]:
fig_ARIMA(df_index=df_SnP_ARIMA, df_test=df_inv_y_test_LO, df_predictions=df_inv_y_pred_test_LO)

# EQUITY CURVE

# Long-Only

# Performance Metrics

In [9]:
position_LO = np.where(df_inv_y_pred_test_LO['inv_y_pred_test'].shift(-1)>df_inv_y_test_LO['inv_y__test'],1,0)

In [10]:
df_Equity_Curve = df_SnP_ARIMA[['Close']]

In [11]:
df_Equity_Curve = df_Equity_Curve[num:len(inv_y_test_LO)+num]

In [12]:
df_Equity_Curve['strat_return'] = df_Equity_Curve['Close'].pct_change().dropna()
df_Equity_Curve['bnh_return'] = df_Equity_Curve['Close'].pct_change().dropna()

In [13]:
def transaction_cost(df_equity_curve, transaction_cost):

    prev_position = df_equity_curve['position'].iloc[0]

    for i, position in enumerate(df_equity_curve['position']):
        if position != prev_position:
            df_equity_curve['strat_return'].iloc[i] -= transaction_cost        
    return df_equity_curve

In [14]:
df_Equity_Curve['position'] = position_LO
df_Equity_Curve = transaction_cost(df_Equity_Curve, TRANSACTION_COST) 

In [15]:
df_Equity_Curve["strategy"] = (df_Equity_Curve["strat_return"] * df_Equity_Curve['position'].shift(1))
df_Equity_Curve["strategy"] = (1+df_Equity_Curve["strategy"].fillna(0)).cumprod()

df_Equity_Curve['buy_n_hold'] = (1 + df_Equity_Curve['bnh_return'].fillna(0)).cumprod()

In [16]:
fig_strategies(df_Equity_Curve)

In [17]:
df_Equity_Curve['position'].value_counts()

0    2510
1    2174
Name: position, dtype: int64

# PERFORMANCE METRICS

In [18]:
wyniki(np.array(df_Equity_Curve['buy_n_hold'].values), 'Equity_Curve_BuyAndHold')

Wyniki dla Equity_Curve_BuyAndHold prezentują się następująco: 

ASD 19.58 
 MD 56.78% 
 ARC 7.53% 
 MLD 1.66 lat 
 IR1 0.3848 
 IR2 0.0511


In [19]:
wyniki(np.array(df_Equity_Curve['strategy'].values), 'Equity_Curve_strategii')

Wyniki dla Equity_Curve_strategii prezentują się następująco: 

ASD 14.45 
 MD 46.73% 
 ARC 1.89% 
 MLD 8.45 lat 
 IR1 0.1307 
 IR2 0.0053


In [20]:
porownanie(np.array(df_Equity_Curve['strategy'].values), np.array(df_Equity_Curve['buy_n_hold'].values))

Strategia lepsza od BH pod względem ASD
 Strategia gorsza od BH pod względem ARC
 Strategia lepsza od BH pod względem MD
 Strategia gorsza od BH pod względem MLD
 Strategia gorsza od BH pod względem IR1
 Strategia gorsza od BH pod względem IR2


### TO CSV - SAVE

In [21]:
STOCK_NAME_FOLDER = f'./Results/{PATH}/Visualisations'

if not os.path.exists(STOCK_NAME_FOLDER):
    os.makedirs(STOCK_NAME_FOLDER)

df_Equity_Curve.to_csv(f'{STOCK_NAME_FOLDER}/df_EC_LO_MAIN.csv')

# Long Short

In [22]:
num = lookback_bars+validation_bars+lag

inv_y_test_LS = df_test_SnP_ARIMA['Close'].values[validation_bars+lag:]
inv_y_pred_test_LS = model_predictions_SnP[validation_bars+lag:]

In [23]:
df_inv_y_test_LS = pd.DataFrame(data={'Date': df_SnP_ARIMA.index[num:(len(inv_y_test_LS)+num)], 'inv_y__test': inv_y_test_LS})
df_inv_y_pred_test_LS = pd.DataFrame(data={'Date': df_SnP_ARIMA.index[num:(len(inv_y_test_LS)+num)], 'inv_y_pred_test': inv_y_pred_test_LS})

In [24]:
fig_ARIMA(df_index=df_SnP_ARIMA, df_test=df_inv_y_test_LS, df_predictions=df_inv_y_pred_test_LS)

# Performance Metrics

In [25]:
# position_LO = (test_model_predictions_LO>0.5).astype(int)
position_LS = np.where(df_inv_y_pred_test_LS['inv_y_pred_test'].shift(-1)>df_inv_y_test_LS['inv_y__test'],1,-1)

In [26]:
df_Equity_Curve_LS = df_SnP_ARIMA[['Close']]

In [27]:
df_Equity_Curve_LS = df_Equity_Curve_LS[num:len(inv_y_test_LS)+num]

In [28]:
df_Equity_Curve_LS['strat_return'] = df_Equity_Curve_LS['Close'].pct_change().dropna()
df_Equity_Curve_LS['bnh_return'] = df_Equity_Curve_LS['Close'].pct_change().dropna()

In [29]:
df_Equity_Curve_LS['position'] = position_LS
df_Equity_Curve_LS = transaction_cost(df_Equity_Curve_LS, TRANSACTION_COST) 

In [30]:
df_Equity_Curve_LS["strategy"] = (df_Equity_Curve_LS["strat_return"] * df_Equity_Curve_LS['position'].shift(1))
df_Equity_Curve_LS["strategy"] = (1+df_Equity_Curve_LS["strategy"].fillna(0)).cumprod()

df_Equity_Curve_LS['buy_n_hold'] = (1 + df_Equity_Curve_LS['bnh_return'].fillna(0)).cumprod()

In [31]:
fig_strategies(df_Equity_Curve_LS)

In [32]:
df_Equity_Curve_LS['position'].value_counts()

-1    2510
 1    2174
Name: position, dtype: int64

# PERFORMANCE METRICS

In [33]:
wyniki(np.array(df_Equity_Curve_LS['buy_n_hold'].values), 'Equity_Curve_BuyAndHold')

Wyniki dla Equity_Curve_BuyAndHold prezentują się następująco: 

ASD 19.58 
 MD 56.78% 
 ARC 7.53% 
 MLD 1.66 lat 
 IR1 0.3848 
 IR2 0.0511


In [34]:
wyniki(np.array(df_Equity_Curve_LS['strategy'].values), 'Equity_Curve_strategii')

Wyniki dla Equity_Curve_strategii prezentują się następująco: 

ASD 19.19 
 MD 54.81% 
 ARC 8.66% 
 MLD 8.44 lat 
 IR1 0.4511 
 IR2 0.0713


In [35]:
porownanie(np.array(df_Equity_Curve_LS['strategy'].values), np.array(df_Equity_Curve_LS['buy_n_hold'].values))

Strategia lepsza od BH pod względem ASD
 Strategia lepsza od BH pod względem ARC
 Strategia lepsza od BH pod względem MD
 Strategia gorsza od BH pod względem MLD
 Strategia lepsza od BH pod względem IR1
 Strategia lepsza od BH pod względem IR2


### TO CSV - SAVE

In [36]:
STOCK_NAME_FOLDER = f'./Results/{PATH}/Visualisations'

if not os.path.exists(STOCK_NAME_FOLDER):
    os.makedirs(STOCK_NAME_FOLDER)

df_Equity_Curve_LS.to_csv(f'{STOCK_NAME_FOLDER}/df_EC_LS_MAIN.csv')